In [ ]:
import numpy as np
import tensorflow as tf
import os
import cv2
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

train_dir = 'E:\\AI\\dataset_skeleton\\training'
val_dir = 'E:\\AI\\dataset_skeleton\\validation'

# 이미지 데이터와 라벨링 데이터를 저장할 리스트 생성
labels = []
images = []

# 주어진 폴더 구조를 순회하며 이미지 데이터와 라벨링 데이터 로드
for root, dirs, files in os.walk(train_dir):
    for dir_name in dirs:        
        class_label = dir_name[:3]  # 폴더 이름의 앞 3자리를 클래스 라벨로 사용
        class_dir = os.path.join(root, dir_name)
        for file in os.listdir(class_dir):
            if file.endswith('.jpg'):
                image_path = os.path.join(class_dir, file)
                image = cv2.imread(image_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                images.append(image)
                labels.append(class_label)
            print(class_dir)

labels = np.array(labels)
print(labels)

import matplotlib.pyplot as plt

# 이미지를 출력하는 함수
def show_images(images, labels, num=5):
    plt.figure(figsize=(15, 15))  # 출력할 전체 프레임의 크기 설정
    for i in range(num):
        plt.subplot(1, num, i+1)  # 1행 num열의 i+1번째 이미지를 출력하는 subplot 설정
        plt.xticks([])  # x축 눈금 제거
        plt.yticks([])  # y축 눈금 제거
        plt.grid(False)
        plt.imshow(images[i])  # 이미지 출력, 이미 RGB 순서로 변경되어 있으므로 그대로 출력
        plt.xlabel(labels[i])  # 이미지 아래에 라벨 출력
    plt.show()

show_images(images, labels)
# 데이터의 개수 확인
data_count = len(images)
print(f'데이터의 총 개수: {data_count}')



E:\AI\dataset_skeleton\training\face
E:\AI\dataset_skeleton\training\face
E:\AI\dataset_skeleton\training\face
E:\AI\dataset_skeleton\training\face
E:\AI\dataset_skeleton\training\face
E:\AI\dataset_skeleton\training\face
E:\AI\dataset_skeleton\training\face
E:\AI\dataset_skeleton\training\face
E:\AI\dataset_skeleton\training\face
E:\AI\dataset_skeleton\training\face
E:\AI\dataset_skeleton\training\face
E:\AI\dataset_skeleton\training\face
E:\AI\dataset_skeleton\training\face
E:\AI\dataset_skeleton\training\face
E:\AI\dataset_skeleton\training\face
E:\AI\dataset_skeleton\training\face
E:\AI\dataset_skeleton\training\side
E:\AI\dataset_skeleton\training\side
E:\AI\dataset_skeleton\training\side
E:\AI\dataset_skeleton\training\side
E:\AI\dataset_skeleton\training\side
E:\AI\dataset_skeleton\training\side
E:\AI\dataset_skeleton\training\side
E:\AI\dataset_skeleton\training\side
E:\AI\dataset_skeleton\training\side
E:\AI\dataset_skeleton\training\side
E:\AI\dataset_skeleton\training\side
E

In [6]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# 라벨 인코딩
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_onehot = to_categorical(labels_encoded)

# 이미지 데이터의 크기 조정 및 정규화
# 예시로 (128, 128) 크기로 조정하고, 255로 나누어 정규화를 진행합니다.
images_resized = np.array([cv2.resize(image, (128, 128)) for image in images]) / 255.0

# CNN 모델 구축
model = Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(np.unique(labels_encoded)), activation='softmax')
])

# 모델 컴파일
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 모델 학습
model.fit(images_resized, labels_onehot, epochs=10, batch_size=32)

# 모델 요약 출력
model.summary()


Epoch 1/10
959/959 [==============================] - 216s 224ms/step - loss: 4.9759 - accuracy: 0.0140
Epoch 2/10
959/959 [==============================] - 215s 224ms/step - loss: 4.0399 - accuracy: 0.0361
Epoch 3/10
959/959 [==============================] - 212s 221ms/step - loss: 3.5760 - accuracy: 0.0661
Epoch 4/10
959/959 [==============================] - 218s 227ms/step - loss: 3.2703 - accuracy: 0.0940
Epoch 5/10
959/959 [==============================] - 218s 227ms/step - loss: 3.0427 - accuracy: 0.1233
Epoch 6/10
959/959 [==============================] - 214s 224ms/step - loss: 2.8573 - accuracy: 0.1504
Epoch 7/10
959/959 [==============================] - 214s 224ms/step - loss: 2.6938 - accuracy: 0.1841
Epoch 8/10
959/959 [==============================] - 225s 235ms/step - loss: 2.5362 - accuracy: 0.2157
Epoch 9/10
959/959 [==============================] - 217s 226ms/step - loss: 2.3874 - accuracy: 0.2477
Epoch 10/10
959/959 [==============================] - 218s 228m

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# ImageDataGenerator 초기화
datagen = ImageDataGenerator(rescale=1./255)  # 이미지를 0과 1 사이의 값으로 정규화

# 훈련 데이터셋을 위한 제너레이터 생성
train_generator = datagen.flow_from_directory(
        data_dir,  # 훈련 데이터셋 디렉토리
        target_size=(224, 224),  # 모든 이미지를 224x224 크기로 조절
        batch_size=32,
        class_mode='categorical')  # 다중 분류 문제인 경우 'categorical'


# 모델 정의
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,  # 검증 데이터셋 설정
    validation_steps=validation_generator.samples // validation_generator.batch_size
)



MemoryError: Unable to allocate 17.2 GiB for an array with shape (30658, 224, 224, 3) and data type float32

In [10]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import clone_model

# 데이터 준비
labels_encoded = label_encoder.fit_transform(labels)
labels_onehot = to_categorical(labels_encoded)

images_resized = []
for image in images:
    resized = cv2.resize(image, (224, 224))
    images_resized.append(resized)

# NumPy 배열로 변환하고 정규화
images_resized = np.array(images_resized, dtype=np.float32) / 255.0

# Stratified K-Fold 교차 검증
n_splits = 5
kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# 교차 검증 점수 기록
scores = []

for train, test in kfold.split(images, labels_encoded):
    # 모델 구축 (모델을 재설정하기 위해 매 반복마다 새로 생성)
    model = Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(len(np.unique(labels_encoded)), activation='softmax')
    ])
    
    # 모델 컴파일
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    # 모델 학습
    model.fit(images[train], labels_onehot[train], epochs=10, batch_size=32, verbose=1)  # verbose=0으로 설정하여 로그 출력을 최소화
    
    # 모델 평가
    scores.append(model.evaluate(images[test], labels_onehot[test], verbose=1))

# 평균 정확도 계산 및 출력
average_accuracy = np.mean([score[1] for score in scores])
print(f'\n평균 정확도: {average_accuracy:.4f}')


MemoryError: Unable to allocate 17.2 GiB for an array with shape (30658, 224, 224, 3) and data type float32